In [419]:
import numpy as np
import pandas as pd
import scipy 
import numpy.matlib
import nilearn
import random
import matplotlib.pyplot as plt
import seaborn as sns
import glob
# import matplotlib_inline
import matplotlib
# matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso, LassoCV, ElasticNet, ElasticNetCV, LinearRegression

In [400]:
# motor and value areas functional connectivity matrix
task_fc = pd.read_csv('task_fc_motor_value.csv', index_col = 'Subject')
X = task_fc

In [401]:
Ys = pd.read_csv('../data/aggregate_indices_subs101_148_08Dec2020.csv', index_col = 'Subject')
Ys = Ys.rename({'AFRIAT Index' : 'Afriat Index', 'VARIAN Index AVG(SSQ)' : 'Varian Index', 'MMI Criterion Avg(SumOfSquares(Wastes))' : 'MMI'}, axis = 1)
Ys = Ys.loc[X.index, :]
measure = 'MMI'
log_measure = 'log' + measure

In [402]:
Ys.loc[:, log_measure] = np.log(Ys[measure])

In [403]:
Y = Ys.loc[:, log_measure]

In [405]:
lasso_mod = LassoCV(alphas=np.linspace(1e-4, 3e-2, 30))
lasso_mod.fit(X, Y)

LassoCV(alphas=array([0.0001    , 0.00113103, 0.00216207, 0.0031931 , 0.00422414,
       0.00525517, 0.00628621, 0.00731724, 0.00834828, 0.00937931,
       0.01041034, 0.01144138, 0.01247241, 0.01350345, 0.01453448,
       0.01556552, 0.01659655, 0.01762759, 0.01865862, 0.01968966,
       0.02072069, 0.02175172, 0.02278276, 0.02381379, 0.02484483,
       0.02587586, 0.0269069 , 0.02793793, 0.02896897, 0.03      ]))

In [416]:
np.corrcoef(Y,lasso_mod.predict(X))

array([[1.        , 0.33205206],
       [0.33205206, 1.        ]])

In [418]:
lasso_mod.alpha_, max(np.linspace(1e-4, 3e-2, 30))

(0.023813793103448277, 0.03)

In [407]:
lasso_selection = lasso_mod.coef_!=0

In [409]:
'+'.join(X.columns[np.where(lasso_mod.coef_!=0)])

'VSTR__SMA+M1__SMA+M1__Kolling+VMPFC__Kolling'

In [452]:
elastic_mod = ElasticNetCV(alphas=[0.1, 0.2], l1_ratio=0, max_iter=1e4)
elastic_mod.fit(X,Y)
elastic_mod.coef_

C:\Users\Asaf\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Asaf\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.74487585406853, tolerance: 0.003980912811568918
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Asaf\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Asaf\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.226590805274242, t

array([-0.07201485, -0.00556279, -0.14205025, -0.03316867,  0.0393993 ,
        0.08248064, -0.00533119,  0.11786932, -0.08970303,  0.12266496,
       -0.05881761, -0.05649931, -0.11319343, -0.1131752 , -0.03333432])

In [425]:
XY_merged = pd.concat([Y, X.iloc[:, lasso_selection]], axis=1)

In [396]:
covariates = '+'.join(XY_merged.columns[1:].to_list())

In [397]:
lm = smf.ols(f'{log_measure} ~ {covariates}', XY_merged)

In [398]:
lm_fit = lm.fit()

In [399]:
print(lm_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 logMMI   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     1.507
Date:                Thu, 10 Feb 2022   Prob (F-statistic):              0.219
Time:                        17:26:02   Log-Likelihood:                -58.301
No. Observations:                  43   AIC:                             126.6
Df Residuals:                      38   BIC:                             135.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -2.6874      1.050     -2.

## Per block

In [337]:
# read trial by trial data and merge into one dataframe
data_dir = r'./trial_data/'
behavioral_files = glob.glob(data_dir+'*')
behavioral_results = [pd.read_excel(behavioral_files[i]) for i in range(len(behavioral_files))]
behavioral_results = pd.concat(behavioral_results)

In [338]:
X = pd.read_csv('task_fc_motor_value_per_block.csv')
X = X.groupby(['Subject', 'block_vec']).mean() # nothing to average, just groupby

In [339]:
pd.options.display.max_columns=100

In [340]:
behavioral_results = behavioral_results.rename({'MMI_In_Sample_component':'MMI', 'SID':'Subject'}, axis=1)
Y = behavioral_results.loc[:, ['Subject', 'obs', 'block', 'MMI', 'slope', 'IntersectionY', 'X_allocation', 'Y_allocation', 'RT', 'SV', 'ChoiceDifficulty']]

In [342]:
Y = Y.groupby(['Subject', 'block']).mean()
Y

obs       MMI     slope  IntersectionY  X_allocation  \
Subject block                                                          
101     1      13.0  0.024680 -1.927825      54.323864        33.812   
        2      38.0  0.019443 -1.618751      56.760247        31.276   
        3      63.0  0.025214 -1.070509      52.305215        47.272   
102     1      13.0  0.015102 -1.604675      47.533694        36.332   
        2      38.0  0.012087 -1.518649      51.181055        36.508   
...             ...       ...       ...            ...           ...   
147     2      38.0  0.038745 -1.648762      62.651625        25.348   
        3      63.0  0.034575 -1.543027      47.735370        40.280   
148     1      13.0  0.003500 -1.561732      53.729655        33.650   
        2      38.0  0.005525 -0.931492      51.985982        46.400   
        3      63.0  0.002613 -2.123862      57.673689        30.704   

               Y_allocation        RT         SV  ChoiceDifficulty  
Subject block                                                       
101     1         32.484000  4.774122  16.330604          0.157050  
        2         36.172000  5.386980  16.310364          0.162391  
        3         26.500000  5.336542  17.315490          0.157008  
102     1         28.912000  6.358332  12.931837          0.125733  
        2         27.812000  6.133793  13.125382          0.087304  
...                     ...       ...        ...               ...  
147     2         38.108000  6.221349  14.182238          0.067616  
        3         25.372000  5.897524  13.674392          0.121429  
148     1         33.416667  8.374055  19.141733          0.214045  
        2         31.300000  7.269716  21.468796          0.206876  
        3         42.836000  5.945575  20.366859          0.288102  

[135 rows x 9 columns]

In [343]:
Y.loc[:, 'logMMI'] = np.log(Y.MMI)

In [363]:
XY_merged = pd.concat([Y, X], axis=1, join='inner')

In [376]:
covariates = '+'.join(X.columns.to_list())
covariates

'SMA__PCC+VSTR__PCC+VSTR__SMA+Kolling__PCC+Kolling__SMA+Kolling__VSTR+M1__PCC+M1__SMA+M1__VSTR+M1__Kolling+VMPFC__PCC+VMPFC__SMA+VMPFC__VSTR+VMPFC__Kolling+VMPFC__M1'

In [371]:
XY_merged = XY_merged.reset_index()
XY_merged = XY_merged.rename({'level_1':'Block'})

In [372]:
lm = smf.mixedlm(f'logMMI ~ {covariates}', XY_merged, groups=XY_merged['Subject'])

In [373]:
lasso_fit = lm.fit_regularized(alpha=3)

In [374]:
lasso_mixed_selection = lasso_fit.params[1:-1]!=0

In [375]:
covariates = '+'.join(X.loc[:, lasso_mixed_selection].columns.to_list())
covariates

'VSTR__PCC+VSTR__SMA+Kolling__SMA+M1__PCC+M1__SMA+M1__VSTR+VMPFC__SMA+VMPFC__VSTR+VMPFC__Kolling'

'VSTR__PCC+VMPFC__VSTR'

In [334]:
lm = smf.mixedlm(f'logMMI ~ {covariates}', XY_merged, groups=XY_merged['Subject'])

In [335]:
lm_fit = lm.fit()

In [336]:
print(lm_fit.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: logMMI   
No. Observations: 128     Method:             REML     
No. Groups:       43      Scale:              0.1396   
Min. group size:  2       Log-Likelihood:     -125.4388
Max. group size:  3       Converged:          Yes      
Mean group size:  3.0                                  
-------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept   -3.974    0.220 -18.095 0.000 -4.405 -3.544
VSTR__PCC   -0.070    0.354  -0.197 0.844 -0.765  0.625
VMPFC__VSTR -0.554    0.341  -1.625 0.104 -1.223  0.114
Group Var    1.170    0.872                            



In [268]:
lm_fit

<bound method LikelihoodModelResults.f_test of <statsmodels.regression.mixed_linear_model.MixedLMResults object at 0x000001BE6FD947F0>>